# ETL cubo

## Preparación de ambiente

### Carga de módulos

In [1]:
# Data Wrangling
import pandas as pd

# Conexión con GCP
from google.oauth2 import service_account

# Environment setup
pd.set_option("display.max_columns", 500)

## Data Wrangling

In [2]:
df = pd.read_csv('./DatosLimpiosSuperclases.csv')
zonas=pd.read_csv('clusters.csv')

df=df.merge(zonas,left_on='dr_no',right_on='dr_no')
df.drop(columns=['zonas6'],axis=1,inplace=True)
df.rename(columns={'zonas5':'zonas'},inplace=True)

In [3]:
df

,dr_no,date_rptd,date_occ,time_occ,area,rpt_dist_no,part_1_2,crm_cd,mocodes,vict_age,vict_sex,vict_descent,premis_cd,weapon_used_cd,status,lat,lon,crm_cd_concat,ucr,premis,zonas
0,10304468,2020-01-08,2020-01-08,1350,3,Others,2,104,0444 0913,36.000000,F,B,501.0,400.0,AO,34.0141,-118.2978,624.0,SIMPLEASSAULT,vivienda,0
1,190101086,2020-01-02,2020-01-01,210,1,Others,2,104,0416 1822 1414,25.000000,M,H,102.0,500.0,IC,34.0459,-118.2545,624.0,SIMPLEASSAULT,calle,0
2,200110444,2020-04-14,2020-02-13,720,1,Others,2,100,1501,23.127650,X,X,726.0,0,AA,34.0448,-118.2474,845.0,Other,Others,0
3,191501505,2020-01-01,2020-01-01,1050,15,Others,2,101,0329 1402,76.000000,F,W,502.0,0,IC,34.1685,-118.4019,745.0998.0,Other,vivienda,1
4,191921269,2020-01-01,2020-01-01,255,19,Others,2,101,0329,31.000000,X,X,409.0,0,IC,34.2198,-118.4468,740.0,Other,establecimiento_publico,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673360,230405171,2023-02-05,2023-02-02,1110,4,Others,1,100,2000 1243 1241 0913 0400 0408 1814,41.000000,F,H,504.0,400.0,IC,34.0808,-118.1738,236.0,AGG. Assults,vivienda,0
673361,231405429,2023-02-05,2023-02-05,770,14,Others,1,103,0325 0344,22.953462,X,X,402.0,0,IC,33.9846,-118.4428,341.0,OTHER THEFT,establecimiento_publico,3
673362,231104474,2023-01-12,2023-01-12,760,11,Others,1,103,1822 0344,42.000000,M,W,101.0,0,IC,34.1214,-118.1915,440.0,OTHER THEFT,calle,0
673363,230804266,2023-01-08,2023-01-08,630,8,Others,1,103,0344 1822,48.000000,M,O,717.0,0,IC,34.0611,-118.4184,341.0,OTHER THEFT,recreacion,3


### Dimensiones

In [4]:
crimens = pd.read_csv('crimecodes_Superclases (1).csv')

In [5]:
dicc_crim = dict(zip(crimens.NewCode, crimens.Superclass))

In [6]:
dicc_crim


{104: 'Robo Vehículo',
 103: 'Robo',
 102: 'Falta a la ley',
 101: 'Disturbios a la sociedad',
 100: 'Agresión'}

In [7]:
df.crm_cd = df.crm_cd.map(dicc_crim)

In [8]:
dicc_status = {'IC':'Investigation',
'AO':'Sanction',
'AA':'Arrest',
'JA':'Arrest',
'JO':'Sanction'}

In [9]:
df.status = df.status.map(dicc_status)
df.status.value_counts()

Investigation    538545
Sanction          74127
Arrest            60693
Name: status, dtype: int64

In [10]:
#sé que dice disc de discretas pero necesitamos estas variables para el tablero gg
ls_disc = ["vict_sex", "zonas", "vict_descent", "premis",'lat','lon','crm_cd','vict_age','status']

### Filtrado de datos

In [11]:
df = df[["date_occ"]+ls_disc].copy()

### Corrección de tipo de dato

In [12]:
df.loc[:, "day"] = pd.to_datetime(df["date_occ"])
df.vict_age = df.vict_age.astype(str) 
df.zonas = df.zonas.astype(str) 
df.lat = df.lat.astype(str) 
df.lon = df.lon.astype(str) 
df.crm_cd = df.crm_cd.astype(str) 

### Reducción de cardinalidad

In [13]:
df = df[["day"]+ls_disc].groupby(["day"]+ls_disc, as_index=False).size()

### Creación de dimensiones temporales

In [14]:
df["week"] = df["day"].dt.to_period('W').dt.start_time

In [15]:
df["month"] = df["day"].dt.to_period('M').dt.start_time

In [16]:
df["quarter"] = df["day"].dt.to_period('Q').dt.start_time

In [17]:
df["year"] = df["day"].dt.to_period('Y').dt.start_time

In [18]:
df

,day,vict_sex,zonas,vict_descent,premis,lat,lon,crm_cd,vict_age,status,size,week,month,quarter,year
0,2020-01-01,F,0,A,Others,34.0487,-118.2627,Robo,22.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
1,2020-01-01,F,0,A,calle,34.1294,-118.2326,Robo Vehículo,30.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
2,2020-01-01,F,0,A,recreacion,34.0309,-118.2981,Robo Vehículo,25.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
3,2020-01-01,F,0,A,recreacion,34.1,-118.3056,Robo Vehículo,62.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
4,2020-01-01,F,0,B,Others,34.0099,-118.3089,Robo Vehículo,23.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668910,2023-02-27,X,4,U,calle,33.7739,-118.2678,Robo Vehículo,30.399084713340493,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668911,2023-02-27,X,4,U,establecimiento_publico,33.7926,-118.3043,Robo,31.818241907545133,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668912,2023-02-27,X,4,U,establecimiento_publico,33.9019,-118.2893,Robo,25.25929147708116,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668913,2023-02-27,X,4,U,recreacion,33.7547,-118.2927,Robo Vehículo,30.446868303197974,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01


In [19]:
df.isna().sum()

day             0
vict_sex        0
zonas           0
vict_descent    0
premis          0
lat             0
lon             0
crm_cd          0
vict_age        0
status          0
size            0
week            0
month           0
quarter         0
year            0
dtype: int64

## Preservación de cubo

### Renombrado de columnas

In [20]:
df = df.rename(columns={"vict_sex": "gender", "zonas": "zone", "vict_descent": "descent", "premis": "place", "size": "crimes","crm_cd":"crime"})

In [22]:
df

,day,gender,zone,descent,place,lat,lon,crime,vict_age,status,crimes,week,month,quarter,year
0,2020-01-01,F,0,A,Others,34.0487,-118.2627,Robo,22.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
1,2020-01-01,F,0,A,calle,34.1294,-118.2326,Robo Vehículo,30.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
2,2020-01-01,F,0,A,recreacion,34.0309,-118.2981,Robo Vehículo,25.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
3,2020-01-01,F,0,A,recreacion,34.1,-118.3056,Robo Vehículo,62.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
4,2020-01-01,F,0,B,Others,34.0099,-118.3089,Robo Vehículo,23.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668910,2023-02-27,X,4,U,calle,33.7739,-118.2678,Robo Vehículo,30.399084713340493,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668911,2023-02-27,X,4,U,establecimiento_publico,33.7926,-118.3043,Robo,31.818241907545133,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668912,2023-02-27,X,4,U,establecimiento_publico,33.9019,-118.2893,Robo,25.25929147708116,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668913,2023-02-27,X,4,U,recreacion,33.7547,-118.2927,Robo Vehículo,30.446868303197974,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01


### Guardado de cubo

### Lectura de credenciales BigQuery

credentials = service_account.Credentials.from_service_account_file(
    './tscd-381619-8e1033c2f61b.json'
)

### Preservación de cubo

In [ ]:
# pip install pandas_gbq
#df.to_gbq(destination_table="crimes.crimes_history", progress_bar=True, if_exists="replace", project_id="tscd-381619", credentials=credentials)

In [ ]:
df

In [26]:
df.to_csv("crimes_cubo.csv")

In [27]:
df

,day,gender,zone,descent,place,lat,lon,crime,vict_age,status,crimes,week,month,quarter,year
0,2020-01-01,F,0,A,Others,34.0487,-118.2627,Robo,22.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
1,2020-01-01,F,0,A,calle,34.1294,-118.2326,Robo Vehículo,30.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
2,2020-01-01,F,0,A,recreacion,34.0309,-118.2981,Robo Vehículo,25.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
3,2020-01-01,F,0,A,recreacion,34.1,-118.3056,Robo Vehículo,62.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
4,2020-01-01,F,0,B,Others,34.0099,-118.3089,Robo Vehículo,23.0,Investigation,1,2019-12-30,2020-01-01,2020-01-01,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668910,2023-02-27,X,4,U,calle,33.7739,-118.2678,Robo Vehículo,30.399084713340493,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668911,2023-02-27,X,4,U,establecimiento_publico,33.7926,-118.3043,Robo,31.818241907545133,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668912,2023-02-27,X,4,U,establecimiento_publico,33.9019,-118.2893,Robo,25.25929147708116,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
668913,2023-02-27,X,4,U,recreacion,33.7547,-118.2927,Robo Vehículo,30.446868303197974,Investigation,1,2023-02-27,2023-02-01,2023-01-01,2023-01-01
